In [1]:
from sentence_transformers import SentenceTransformer

model_name = "hiieu/halong_embedding"

model = SentenceTransformer('hiieu/halong_embedding')


/home/mahdi/text_embedding/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sentence_transformers import InputExample
import csv

train_examples = []
with open('persian_pairs.csv', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        # assume columns: text1, text2, score (float 0–1)
        train_examples.append(
            InputExample(texts=[row['text1'], row['text2']],
                         label=float(row['score']))
        )



In [10]:
from datasets import load_from_disk
train_dataset = load_from_disk('APN_dataset')

In [11]:
# print first 5 examples
for ex in train_dataset.select(range(5)):
    print(ex)


{'anchor': 'عقد مرابحه یک نوع عقد بانکی است که در آن بانک برای مبلغ وام\u200cدهی، به برداشتن شخص قابل امتیاز تشخیص می\u200cدهد که باید برای این مقدار دریافت شده چقدر سود بپردازد. این اطلاعات به صورت منظم و با روشنی تعیین می\u200cشود، که نتیجه آن شفافیت در قرارداد است.', 'positive': 'عقد مرابحه چیست و آیا شفاف بودن قرارداد اصلی در این نوع عقد با اشاره به مبلغ سود است؟', 'negetive': 'چه نوع از وام\u200cهای ازدواجی توصیه می\u200cشود و آیا آن\u200cها در این منبع تحت پوشش هستند؟'}
{'anchor': 'بله، به نقطه تاس۱۰ توجه کنید: «امکان ضمانت متقابل انواع وام\u200cها در بانک ملت وجود ندارد».', 'positive': 'آیا امکان ضمانت متقابل در وام\u200cهای مختلف در بانک ملت وجود دارد؟', 'negetive': 'نحوه محاسبه توان بازپرداخت چگونه است؟'}
{'anchor': 'قرض الحسنه یعنی وامی است که بدون سود و بهره داده می\u200cشود و فقط اصل پول باید برگردانده شود.', 'positive': 'قرض الحسنه به معنای کدام نوع وام است؟', 'negetive': 'وام های ازدواج و فرزندآوری در طبقه\u200cبندی معمول تکلیفی قرار می\u200cگیرند یا نه؟'}
{'anchor': 'وثی

In [3]:
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_loader = DataLoader(train_examples, shuffle=True, batch_size=2)

# For supervised scores:
train_loss = losses.CosineSimilarityLoss(model)
# Or, for contrastive learning / mining:
# train_loss = losses.MultipleNegativesRankingLoss(model)


In [4]:
from sentence_transformers import InputExample

# Example: 5 held-out pairs
dev_examples = [
    InputExample(texts=["من فوتبال دوست دارم", "من عاشق فوتبالم"],  label=0.9),
    InputExample(texts=["این ماشین سریع است", "این ماشین آهسته است"], label=0.1),
    # … add 30–100 examples for reliability
]


In [5]:
from sentence_transformers import evaluation

sts_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(
    dev_examples, 
    name='persian-sts'
)


In [6]:
model.fit(
    train_objectives=[(train_loader, train_loss)],
    epochs=3,
    warmup_steps=100,
    evaluator=sts_evaluator,         # or ir_evaluator
    evaluation_steps=200,            # run eval every 200 steps
    output_path='./halong_persian_finetuned'
)


Step,Training Loss,Validation Loss,Persian-sts Pearson Cosine,Persian-sts Spearman Cosine
6,No log,No log,-1.000000,-1.000000
12,No log,No log,1.000000,1.000000
18,No log,No log,1.000000,1.000000


In [7]:
model.save('halong_persian_finetuned')

In [9]:
from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer('./halong_persian_finetuned')
model = SentenceTransformer('hiieu/halong_embedding')
pairs = [
    ("امروز هوا خوب است", "هوا امروز عالیه"),
    ("من کتاب می‌خوانم", "من در حال مطالعه کتاب هستم"),
    ("این ماشین سریع است", "این ماشین آهسته است"),
]

# Encode all sentences at once
embeddings = model.encode(
    [s for pair in pairs for s in pair],
    convert_to_tensor=True
)

for i, (a, b) in enumerate(pairs):
    sim_tensor = util.cos_sim(embeddings[2*i], embeddings[2*i+1])
    # sim_tensor is a 1×1 Tensor, so .item() gives the float inside
    sim_score = sim_tensor.item()
    print(f"{a!r} ≅ {b!r} → cosine-sim = {sim_score:.4f}")


'امروز هوا خوب است' ≅ 'هوا امروز عالیه' → cosine-sim = 0.8102
'من کتاب می\u200cخوانم' ≅ 'من در حال مطالعه کتاب هستم' → cosine-sim = 0.8366
'این ماشین سریع است' ≅ 'این ماشین آهسته است' → cosine-sim = 0.9105
